関数型のスタイルでプログラミングをする場合、イテレータが重要な基礎となる。  
- イテレータとは
  - 連続データを表現するオブジェクト
  - 一度に一つづつ要素を返す
  - 引数を取らずに次の要素を返すメソッドを必ずサポートしている
    - `__next__()`
      - データストリームに要素が残っていなければStopIteration例外を返す
  - 必要に応じて無限のイテレータを書くこともある



- ビルトインのiter()
  - 任意のオブジェクトを受け取り、その中身や要素を返すイテレータを返す。
  - 渡したオブジェクトがイテレータを作れないものであった場合はTypeErrorを返す
    - リストや辞書などのイテラブルなオブジェクト

In [15]:
L = [1,2,3]
it = iter(L)
it

In [16]:
for i in it:
    print(i)

1
2
3


前のセルでオブジェクトの中身を使い果たしているのでStopIteration

In [17]:
next(it)

StopIteration: 

イテレータはコンストラクタ関数でリストやタプルに具現化できる


In [22]:
iterator = iter(L)
t = tuple(iterator)
t

(1, 2, 3)

In [26]:
# アンパック
a,b,c, = t
a,b,c

(1, 2, 3)

シーケンス型はどれでも自動でイテレータ生成に対応している

In [27]:
# 3.7から辞書の反復順序は挿入順序と同じで有ることが保証された
m = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12,
}
for key in m:
    print(key, m[key])

Jan 1
Feb 2
Mar 3
Apr 4
May 5
Jun 6
Jul 7
Aug 8
Sep 9
Oct 10
Nov 11
Dec 12


辞書はキーでイテレータを回すが、明示的に値やキーあるいはそれらのペアでイテレートしたい場合は`values()`や`items()`メソッドでイテレータを作ることができる。


`dict()`は`(key,value)`タプルのストリームを返すイテレータを受け入れることができる。

In [28]:
l = [("Italy", "Rome"), ("France", "Paris"), ("US", "Washington DC")]
dict(iter(l))

{'Italy': 'Rome', 'France': 'Paris', 'US': 'Washington DC'}

セットもイテラブルを受け取れるし、要素でイテレートできる。

In [30]:

S = {2,3,5,7,11,13}
for i in S:
    print(i)

2
3
5
7
11
13


## ジェネレータ式とリスト内包表記
- イテレータによく使う操作
  - 一つづつ全要素に操作を実行する
  - 条件の合う要素でサブセットを作る



下記は文字列のストリームから空白を取り除くコード

In [32]:
line_list = ["   line 1\n", "line 2   \n", "  \n", ""]


In [35]:
from typing import Generator


striped_iter: Generator[str, None, None] = (line.strip() for line in line_list)

<generator object <genexpr> at 0x10bd632a0>

In [36]:
striped_list: list[str] = [line.strip() for line in line_list]

In [38]:
# if条件式で特定の要素を抜き取る
striped_list2: list[str] = [line.strip() for line in line_list if line == ""]

In [41]:
seq1 = "abc"
seq2 =(1,2,3)
seq_marge = [(x,y) for x in seq1 for y in seq2]
seq_marge[0]

('a', 1)